In [1]:
import json
import subprocess

import tempfile

tempfile.tempdir = "/tmp/colima"

import tiktoken

enc = tiktoken.encoding_for_model("gpt-4")

with open("errors.json", "r") as f:
    errors = json.load(f)

    commit_hash = "b6a48a6e557fad1ceda680618e0a34c7b8c5c087"
    commit = errors[commit_hash]

    repo_slug = commit["input"]["repo_slug"]
    file_in_scope = commit["input"]["file_in_scope"]

    repo_dir = tempfile.mkdtemp()

    extracted_diff = commit["raw_output"]

    reproduction_command = "mvn -X -Dsurefire.printSummary=true -Dsurefire.redirectTestOutputToFile=false clean test -B"

commit

{'patch': '```diff\nPath: nis/src/main/java/org/nem/specific/deploy/appconfig/NisAppConfig.java\n@@ - final org.flywaydb.core.Flyway flyway = new Flyway();\n- flyway.setDataSource(this.dataSource());\n- flyway.setClassLoader(NisAppConfig.class.getClassLoader());\n- flyway.setLocations(prop.getProperty("flyway.locations"));\n- flyway.setValidateOnMigrate(Boolean.valueOf(prop.getProperty("flyway.validate")));\n+ final org.flywaydb.core.Flyway flyway = Flyway.configure()\n+ .dataSource(this.dataSource())\n+ .classLoader(NisAppConfig.class.getClassLoader())\n+ .locations(prop.getProperty("flyway.locations"))\n+ .validateOnMigrate(Boolean.valueOf(prop.getProperty("flyway.validate")))\n+ .load();\n```',
 'error': 'Error during compilation: [ERROR] Failed to execute goal org.apache.maven.plugins:maven-surefire-plugin:3.1.0:test (default-test) on project nem-core:\n[ERROR] Please refer to /mnt/repo/core/target/surefire-reports for the individual test results.\n[ERROR] Please refer to dump file

In [2]:
from pathlib import Path

from masterthesis.agent.LSPAgent import LSPAgent
from masterthesis.agent.aider.AdvancedDiffAgent import UnifiedDiffCoder
from masterthesis.llm.pipeline import pipeline


def invalid_diff_callback(is_valid_diff, diff_remarks):
    # candidate_results[commit_hash]['patch'] = extracted_diff
    if not is_valid_diff:
        print(
            f"The Diff is not valid for the given source code. The tool exited with: {diff_remarks}",
            flush=True,
        )
        raise ValueError("The Diff is not valid for the given source code")
    print("The Diff is valid", flush=True)
    # print(diff_remarks[:2000], flush=True)


def diagnostic_callback(diagnostics):
    print("Initial diagnostics", flush=True)
    print("-" * 20, flush=True)

    print(diagnostics, flush=True)

    print("-" * 20, flush=True)

    # dspy.Suggest(len(diagnostics) > 0, f"Linting failed with the following diagnostics: {diagnostics}")
    # if len(diagnostics) == 0:
    #     print("Linting passed", flush=True)
    # else:
    #     print("Suggested", diagnostics, flush=True)


def compile_callback(has_errors, error_text):
    # dspy.Assert(not has_errors, f"Error during compilation: {error_text}")
    if has_errors:
        print(f"Error during compilation: {error_text}", flush=True)
    else:
        print("Compilation passed", flush=True)


def generate_diff_callback(
    code, path, api_changes, initial_error, updated_dependency_diff
):
    # candidate_results[commit_hash]['input']['prepared_file'] = test_code_contents
    # candidate_results[commit_hash]['final_assumed_token_count'] = token_count_file + param_token_count

    # extracted_diff = self.generate_diff(code=prepared_file, path=file_in_scope, api_changes=api_changes, initial_error=initial_error_lines, updated_dependency_diff=updated_dependency_diff).answer
    # candidate_results[commit_hash]['raw_output'] = extracted_diff
    return extracted_diff


pipeline(
    0,
    file_in_scope,
    0,
    extracted_diff,
    0,
    repo_slug,
    commit_hash,
    reproduction_command,
    invalid_diff_callback,
    diagnostic_callback,
    compile_callback,
    generate_diff_callback,
)

The Diff is valid
Pulling image ghcr.io/lukvonstrom/multilspy-java-docker:latest...
Pulling image ghcr.io/lukvonstrom/multilspy-java-docker:latest...
Started container
Executing command /bin/bash -c '/app/init.sh -e yes -f nis/src/main/java/org/nem/specific/deploy/appconfig/NisAppConfig.java'
Initial diagnostics
--------------------
['[JAVA] 109:3 to 109:14 - The method classLoader(ClassLoader) is undefined for the type FluentConfiguration', '[JAVA] 109:3 to 109:14 - The method classLoader(ClassLoader) is undefined for the type FluentConfiguration', '[JAVA] 109:3 to 109:14 - The method classLoader(ClassLoader) is undefined for the type FluentConfiguration']
--------------------
Error during compilation: [ERROR] Tests run: 8, Failures: 0, Errors: 4, Skipped: 0, Time elapsed: 0.858 s <<< FAILURE! - in org.nem.core.crypto.KeyPairTest
[ERROR] org.nem.core.crypto.KeyPairTest.ctorCreatesKeyGenerator  Time elapsed: 0.269 s  <<< ERROR!
[ERROR] org.nem.core.crypto.KeyPairTest.ctorWithPrivateKey

'```diff\nPath: nis/src/main/java/org/nem/specific/deploy/appconfig/NisAppConfig.java\n@@ - final org.flywaydb.core.Flyway flyway = new Flyway();\n- flyway.setDataSource(this.dataSource());\n- flyway.setClassLoader(NisAppConfig.class.getClassLoader());\n- flyway.setLocations(prop.getProperty("flyway.locations"));\n- flyway.setValidateOnMigrate(Boolean.valueOf(prop.getProperty("flyway.validate")));\n+ final org.flywaydb.core.Flyway flyway = Flyway.configure()\n+ .dataSource(this.dataSource())\n+ .classLoader(NisAppConfig.class.getClassLoader())\n+ .locations(prop.getProperty("flyway.locations"))\n+ .validateOnMigrate(Boolean.valueOf(prop.getProperty("flyway.validate")))\n+ .load();\n```'